In [1]:
import os
import json
import pandas as pd
import traceback



  Obtaining dependency information for PyPDF2 from https://files.pythonhosted.org/packages/8e/5e/c86a5643653825d3c913719e788e41386bee415c2b87b4f955432f2de6b2/pypdf2-3.0.1-py3-none-any.whl.metadata
Using cached pypdf2-3.0.1-py3-none-any.whl (232 kB)


In [7]:
from langchain.chat_models import ChatOpenAI


In [4]:
from dotenv import load_dotenv
load_dotenv()
KEY = os.getenv("OPENAI_API_KEY")

llm = ChatOpenAI(openai_api_key=KEY, model_name="gpt-3.5-turbo", temperature=0.5)

/Users/yujunchen/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [11]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2
 


In [34]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
         "options": {
           "a": "choice here",
             "b": "choice here",
              "c": "choice here",
              "d": "choice here",
           },
    "correct": "correct answer",
       },
    "2": {
        "mcq": "multiple choice question",
         "options": {
           "a": "choice here",
             "b": "choice here",
              "c": "choice here",
              "d": "choice here",
           },
    "correct": "correct answer",
         },
   "3": {
        "mcq": "multiple choice question",
         "options": {
           "a": "choice here",
             "b": "choice here",
              "c": "choice here",
              "d": "choice here",
           },
    "correct": "correct answer",
         },
  },


TEMPLATE1 = """
    Text: {text}
    You are an expert MCQ maker. Given the above text, it is your job to \
    create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. \
    Make sure the questions are not repeated and check all the questions to be conforming the text as well.\
    Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
    Ensure to make {number} MCQs 
    ### RESPONSE_JSON
    {response_json}
    """
quiz_gen_prompt = PromptTemplate(input_variables=["text","number", "subject", "tone", "response_json"], template=TEMPLATE1)

In [21]:
quiz_chain = LLMChain(llm=llm, prompt=quiz_gen_prompt, output_key="quiz",verbose= True)

In [23]:
TEMPLATE2 = """ 
    You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students, \
    You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis,\
    if the quiz is not at per with the cognitive and analytical abilities of the students,\
    update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities,\
    Quiz_MCQs:
    {quiz}
    
    Check from an expert English Writer of the above quiz:
 """

In [24]:
quiz_eval_prompt = PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)


In [25]:
review_chain = LLMChain(llm=llm, prompt=quiz_eval_prompt, output_key="review",verbose= True)

In [29]:
generate_evaluate_chain = SequentialChain(chains=[quiz_chain, review_chain],input_variables=["text", "number", "subject", 'tone','response_json'], output_variables=['quiz', 'review'],verbose=True)

In [32]:
file_path = "/Users/yujunchen/Desktop/AI/AI-Projects/mcqgen/data.txt"
with open(file_path, 'r') as file:
    TEXT = file.read()


 The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[9][10] The synonym self-teaching computers was also used in this time period.[11][12]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[14] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[13] Other researchers who have studied human cognitive systems contribute

In [109]:
# Serialize the Python dictionary into a JSON-formatted string
RESPONSE_JSON

'[{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}]'

In [110]:
NUMBER=5
SUBJECT="Machine Learning"
TONE="simple"


In [111]:
#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response = generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject": SUBJECT,
            "tone" : TONE,
            "response_json":json.dumps(RESPONSE_JSON)
        }
    )





> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

    Text:  The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[9][10] The synonym self-teaching computers was also used in this time period.[11][12]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[14] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons

In [112]:
print(f"Total Tokens Used: {cb.total_tokens}")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total cost: {cb.total_cost} USD")

Total Tokens Used: 1977
Prompt Tokens: 1478
Completion Tokens: 499
Total cost: 0.0032150000000000004 USD


In [119]:
quiz=response['quiz']
quiz = json.dumps(RESPONSE_JSON)
print(quiz)

"[{\"1\": {\"mcq\": \"multiple choice question\", \"options\": {\"a\": \"choice here\", \"b\": \"choice here\", \"c\": \"choice here\", \"d\": \"choice here\"}, \"correct\": \"correct answer\"}, \"2\": {\"mcq\": \"multiple choice question\", \"options\": {\"a\": \"choice here\", \"b\": \"choice here\", \"c\": \"choice here\", \"d\": \"choice here\"}, \"correct\": \"correct answer\"}, \"3\": {\"mcq\": \"multiple choice question\", \"options\": {\"a\": \"choice here\", \"b\": \"choice here\", \"c\": \"choice here\", \"d\": \"choice here\"}, \"correct\": \"correct answer\"}}]"


In [120]:
quiz_table_data = []
for key,value in quiz.items():
    mcq = value['mcq']
    options = " | ".join([f"{option}: {option_value}" for option,option_value in value['options'].items()]
                         )
    correct = value['correct']
    quiz_table_data.append([mcq, options, correct])

AttributeError: 'str' object has no attribute 'items'